# Выбор локации для скважины

Поставленная задача:

Есть пробы нефти в трех регионах (качество и объемы запасов, по 10000 месторождений). На основании этих данных необходимо построить модель машинного обучения, которая поможет определить регион, где добыча нефти принесет наибольшую прибыль.

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from numpy.random import RandomState
state = RandomState(25)

## Загрузка и подготовка данных

Загрузим данные.

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Посмотрим на состояние данных в таблицах.

In [3]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


С данными всё в порядке. Подготовим данные для модели. 

## Обучение и проверка модели

Разделим выборки на тестовые и валидоционные, масштабируем данные.

In [9]:
features_0 = data_0.drop(['id', 'product'], axis=1)
target_0 = data_0['product']
features_1 = data_1.drop(['id', 'product'], axis=1)
target_1 = data_1['product']
features_2 = data_2.drop(['id', 'product'], axis=1)
target_2 = data_2['product']

In [10]:
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(
        features_0, target_0, test_size=0.25, random_state=25)
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(
        features_1, target_1, test_size=0.25, random_state=25)
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(
        features_2, target_2, test_size=0.25, random_state=25)

In [11]:
model = LinearRegression()
model.fit(features_0_train, target_0_train)
predicted_0 = pd.Series(model.predict(features_0_valid), index=features_0_valid.index)
model.fit(features_1_train, target_1_train)
predicted_1 = pd.Series(model.predict(features_1_valid), index=features_1_valid.index)
model.fit(features_2_train, target_2_train)
predicted_2 = pd.Series(model.predict(features_2_valid), index=features_2_valid.index)

In [12]:
rmse_0 = mean_squared_error(target_0_valid, predicted_0) ** 0.5
rmse_1 = mean_squared_error(target_1_valid, predicted_1) ** 0.5
rmse_2 = mean_squared_error(target_2_valid, predicted_2) ** 0.5

Сравним основные параметры по регионам.

Регион 0

In [13]:
print(f'Средний запас сырья по предсказаниям: {predicted_0.mean()}')
print(f'средний запас сырья по правильным ответам: {target_0_valid.mean()}')
print(f'RMSE модели: {rmse_0}')

Средний запас сырья по предсказаниям: 92.65108756805346
средний запас сырья по правильным ответам: 92.40208190394397
RMSE модели: 37.65053057971891


Регион 1

In [14]:
print(f'Средний запас сырья по предсказаниям: {predicted_1.mean()}')
print(f'средний запас сырья по правильным ответам: {target_1_valid.mean()}')
print(f'RMSE модели: {rmse_1}')

Средний запас сырья по предсказаниям: 69.27401526922849
средний запас сырья по правильным ответам: 69.27618376748285
RMSE модели: 0.8892646983510678


Регион 2

In [15]:
print(f'Средний запас сырья по предсказаниям: {predicted_2.mean()}')
print(f'средний запас сырья по правильным ответам: {target_2_valid.mean()}')
print(f'RMSE модели: {rmse_2}')

Средний запас сырья по предсказаниям: 94.90290511535086
средний запас сырья по правильным ответам: 95.02246484619852
RMSE модели: 40.08029177014584


Результаты работы модели на выборке региона 1 значительно отличаются от двух других. Модель в этом случае случае получила достаточно точные значения, RMSE меньше единицы.

В выборках регионов 0 и 2 RMSE отличается не очень сильно. Предсказания в этих выборках не так точны.

## Подготовка к расчёту прибыли

Создадим переменные для основных данных в расчетах.

In [16]:
budget = 10000 # 10 млрд - 10 000 млн рублей
income = 0.450 # 450 тысяч с каждой тысячи баррелей, т.е. с единицы продукта, 0.45 млн. рублей
wells = 200 # 200 скважин должно быть в разработке
budget_well = budget / wells # тыс. рублей на разработку одной скважины

посчитаем количество сырья для безубыточной разработки скважины.

In [17]:
budget_well / income

111.11111111111111

Для безубыточной разработки объем сырья в скважине должен превышать 111 единиц. Это выше среднего для всех трех регионов.

## Расчёт прибыли и рисков 

Напишем функцию для вычисления потенциальной прибыли региона по предсказаниям модели.

In [29]:
def income_region(predictions, target):
    best_200_predictions = predictions.sort_values(ascending=False)[:200]
    best_200_target = target[best_200_predictions.index]
    total_income = best_200_target.sum() * income - budget
    return total_income

In [19]:
f'Максимальная прибыль в регионе 0: {income_region(predicted_0, target_0_valid) / 1000:.2f} млрд. рублей'

'Максимальная прибыль в регионе 0: 3.28 млрд. рублей'

In [20]:
f'Максимальная прибыль в регионе 1: {income_region(predicted_1, target_1_valid) / 1000:.2f} млрд. рублей'

'Максимальная прибыль в регионе 1: 2.42 млрд. рублей'

In [21]:
f'Максимальная прибыль в регионе 2: {income_region(predicted_2, target_2_valid) / 1000:.2f} млрд. рублей'

'Максимальная прибыль в регионе 2: 2.46 млрд. рублей'

Прибыль от разработки скважин во всех регионах положительная. Наибольшая в регионе "0".

Применим технику Bootstrap, чтобы найти распределение прибыли.

In [22]:
incomes_0 = []
incomes_1 = []
incomes_2 = []

for i in range(1000):
    predicted_0_bs = predicted_0.sample(n=500, replace=True, random_state=state)
    incomes_0.append(income_region(predicted_0_bs, target_0_valid))
    predicted_1_bs = predicted_1.sample(n=500, replace=True, random_state=state)
    incomes_1.append(income_region(predicted_1_bs, target_1_valid))
    predicted_2_bs = predicted_2.sample(n=500, replace=True, random_state=state)
    incomes_2.append(income_region(predicted_2_bs, target_2_valid))

incomes_0 = pd.Series(incomes_0)
incomes_1 = pd.Series(incomes_1)
incomes_2 = pd.Series(incomes_2)

Для каждого региона посмотрим на среднюю прибыль, 95%-ый доверительный интервал и риск убытков.

In [31]:
confidence_interval = st.t.interval(alpha=0.95, df=999, loc=incomes_0.mean(), scale=incomes_0.sem())
print('Регион "0"')
print(f'Среднее значение прибыли: {incomes_0.mean():.2f} млн. рублей')
print(f'95%-ый доверительный интервал: {incomes_0.quantile(0.025)} - {incomes_0.quantile(0.975)} млн. рублей')
print(f'Риск убытков: {np.mean(incomes_0 < 0)}')

Регион "0"
Среднее значение прибыли: 362.46 млн. рублей
95%-ый доверительный интервал: -125.42155515649911 - 841.2677197792207 млн. рублей
Риск убытков: 0.077


In [33]:
confidence_interval = st.t.interval(alpha=0.95, df=999, loc=incomes_1.mean(), scale=incomes_1.sem())
print('Регион "1"')
print(f'Среднее значение прибыли: {incomes_1.mean():.2f} млн. рублей')
print(f'95%-ый доверительный интервал: {incomes_1.quantile(0.025)} - {incomes_1.quantile(0.975)} млн. рублей')
print(f'Риск убытков: {np.mean(incomes_1 < 0)}')

Регион "1"
Среднее значение прибыли: 502.25 млн. рублей
95%-ый доверительный интервал: 120.90027580857527 - 896.4964302813606 млн. рублей
Риск убытков: 0.006


In [34]:
confidence_interval = st.t.interval(alpha=0.95, df=999, loc=incomes_2.mean(), scale=incomes_2.sem())
print('Регион "2"')
print(f'Среднее значение прибыли: {incomes_2.mean():.2f} млн. рублей')
print(f'95%-ый доверительный интервал: {incomes_2.quantile(0.025)} - {incomes_2.quantile(0.975)} млн. рублей')
print(f'Риск убытков: {np.mean(incomes_2 < 0)}')

Регион "2"
Среднее значение прибыли: 344.24 млн. рублей
95%-ый доверительный интервал: -203.00870210689502 - 843.9809036403182 млн. рублей
Риск убытков: 0.103


Наименьший риск убытков получен в регионе "1", там же нижняя граница доверительного интервала положительная в отличие от других регионов. Средняя прибыль также выше.

Таким образом, регион "1" является наилучшим для разработки.